# Module 3: Feature Selection

In this lab you will learn about **feature selection**, 
which reduces the dimensionality of data for the following reasons:

1. Reduces overfitting by removing noise introduced by some of the features.
2. Reduces training time, which allows you to experiment more with different models and hyperparameters.
3. Reduces data acquisition requirements.
4. Improves comprehensibility of the model because a smaller set of features is more comprehendible to humans. That will enable you to focus on the main sources of predictability, make the model more justifiable to another person.

For this session, we are going to use **red wine quality** dataset as a starting point for learning feature selection,
and then select the most relevant feature for predicting wine quality.



Feature selection methods generally falls into two categories known as **wrapper methods** and **filter methods**. There is a 3rd category, **embedded methods**, which includes those models where features are identified while learning the model parameters (e.g., LASSO). In this notebook we will discuss wrapper and filter methods.


Wrappers utilize the learning machine of interest as a black box to score subsets of
variable according to their predictive power. Filters select subsets of variables as a pre-processing
step, independently of the chosen predictor. Embedded methods perform v


<span style="text-decoration: underline;">**Filter methods**</span> apply a statistical measure and assign a score to each feature one at a time. These methods select features independent of predictive methods and run very fast. 
In this lab, you will go through **Pearson's χ²** and **ANOVA F-value based feature selection** in this section. 


<span style="text-decoration: underline;">**Wrapper methods**</span> use predictive methods for selecting a subset of features. The idea is to learn a set of features with feedbacks from the predictive method of interest. Based on the results drawn from the predictive model trained on that subset of features, 
they are either added or removed from the subset.
The problem is essentially reduced to a search problem.
[Greedy algorithms](https://en.wikipedia.org/wiki/Greedy_algorithm) 
are the most desirable in multivariate feature selection scenario because 1) 
wrapper methods are usually computationally very expensive; 2) 
greedy algorithms don't necessary provide the optimal solution,
which is good becuase it makes them less prone to overfitting.



sklearn API reference:

+ [sklearn.feature_selection.SelectKBest](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html)
+ [sklearn.feature_selection.chi2](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.chi2.html)
+ [sklearn.feature_selection.f_regression](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_regression.html)
+ [sklearn.feature_selection.mutual_info_classif](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_classif.html)
+ [sklearn.feature_selection.RFE](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)



In [1]:
import os, sys
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import RFE

from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB

from sklearn.base import clone

# Uncomment following line to view original output.
# np.random.seed(18937)

## Load Dataset

In [2]:
# Dataset location
DATASET = '/dsa/data/all_datasets/wine-quality/winequality-red.csv'
assert os.path.exists(DATASET)

# Load and shuffle
dataset = pd.read_csv(DATASET, sep=';').sample(frac = 1).reset_index(drop=True)
dataset.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


Store features and labels into variables **X** and **y** respectively.

In [3]:
X = dataset.iloc[:, :-1].to_numpy()
y = dataset.quality

## Feature selection solution space

From algorithm analysis point of view, a solution to feature selection problems can be respresented as a boolean vector,
each component indicating whether the corresponding feature has been selected.
For instance,

In [4]:
selected = np.array([False, True, True, False, False, True, True, False, False, False, True])

sci-kit learn calls the corresponding indices to feature columns selected "support", 
which can be obtained using [np.flatnonzero()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.flatnonzero.html):

In [5]:
support = np.flatnonzero(selected)
print(support)

[ 1  2  5  6 10]


Thus a naive approach that exhaustively search all subsets of features would have to verify $2^p$ solutions for $p$ features,
which translates into $\Omega(2^p)$ [time complexity](https://en.wikipedia.org/wiki/Time_complexity).
That is to say it would be very inefficient in practice.

However, we will run an exhaustive search for all solutions that provide 5 features to establish a baseline.
This limits time complexity to $O(p^5 \cdot n)$, assuming scoring a model takes linear time $O(n)$.
Therefore, the following cell checks all $\begin{pmatrix} 11 \\ 5 \end{pmatrix} = \frac{11\times10\times9\times8\times7}{5\times4\times3\times2\times1}=462$ solutions, 
and displays top 3 subset of features ranked by accuracy.

In Part 1 "Wrapper methods", we will use these solutions as comparison.

In [6]:
import itertools

def search_combinations(estimator, X, y, k=5):
    # fit and score model based on some subset of features
    score = lambda X_features: clone(estimator).fit(X_features, y).score(X_features, y)
    
    # enumerate all combinations of 5 features
    for subset in itertools.combinations(range(X.shape[1]), 5):
        print(subset)
        sc = score(X[:, subset])
        print(sc)
        yield score(X[:, subset]), subset
        
sorted(search_combinations(LinearRegression(), X, y), reverse=True)

(0, 1, 2, 3, 4)
0.16586122214533672
(0, 1, 2, 3, 5)
0.15635743420404236
(0, 1, 2, 3, 6)
0.17959756352193668
(0, 1, 2, 3, 7)
0.2334643033929913
(0, 1, 2, 3, 8)
0.15804082175121648
(0, 1, 2, 3, 9)
0.17782253810278803
(0, 1, 2, 3, 10)
0.3247565593351991
(0, 1, 2, 4, 5)
0.16814241289738752
(0, 1, 2, 4, 6)
0.19035466684200242
(0, 1, 2, 4, 7)
0.22331297243055537
(0, 1, 2, 4, 8)
0.16734971452876723
(0, 1, 2, 4, 9)
0.21042352498980965
(0, 1, 2, 4, 10)
0.3248472412972452
(0, 1, 2, 5, 6)
0.18307405978739355
(0, 1, 2, 5, 7)
0.2191795415852923
(0, 1, 2, 5, 8)
0.16025672103652366
(0, 1, 2, 5, 9)
0.18136126956519405
(0, 1, 2, 5, 10)
0.3248208013891577
(0, 1, 2, 6, 7)
0.2304021064556867
(0, 1, 2, 6, 8)
0.17933763697874539
(0, 1, 2, 6, 9)
0.2042698853529391
(0, 1, 2, 6, 10)
0.3283177236868041
(0, 1, 2, 7, 8)
0.2335519471866464
(0, 1, 2, 7, 9)
0.2528146541206694
(0, 1, 2, 7, 10)
0.3249257461332231
(0, 1, 2, 8, 9)
0.1837066228280554
(0, 1, 2, 8, 10)
0.32662545415489586
(0, 1, 2, 9, 10)
0.343935296701131

0.3362407365865345
(1, 3, 8, 9, 10)
0.3389081628512486
(1, 4, 5, 6, 7)
0.20853991518748705
(1, 4, 5, 6, 8)
0.19092569439364138
(1, 4, 5, 6, 9)
0.23989650872722268
(1, 4, 5, 6, 10)
0.3248057695323059
(1, 4, 5, 7, 8)
0.19055504563340886
(1, 4, 5, 7, 9)
0.24312573554257122
(1, 4, 5, 7, 10)
0.3199355739042917
(1, 4, 5, 8, 9)
0.21452231713658998
(1, 4, 5, 8, 10)
0.32451290112611453
(1, 4, 5, 9, 10)
0.3443635711764268
(1, 4, 6, 7, 8)
0.20872861033509504
(1, 4, 6, 7, 9)
0.2616956810928466
(1, 4, 6, 7, 10)
0.3253168666926135
(1, 4, 6, 8, 9)
0.23645144990985134
(1, 4, 6, 8, 10)
0.3308391214739568
(1, 4, 6, 9, 10)
0.3514942385018405
(1, 4, 7, 8, 9)
0.24009667865863127
(1, 4, 7, 8, 10)
0.32486985548061675
(1, 4, 7, 9, 10)
0.343362604424549
(1, 4, 8, 9, 10)
0.3483140356739335
(1, 5, 6, 7, 8)
0.20584709238437626
(1, 5, 6, 7, 9)
0.24025382103484594
(1, 5, 6, 7, 10)
0.32631187096938186
(1, 5, 6, 8, 9)
0.2095180258485465
(1, 5, 6, 8, 10)
0.33183948108138606
(1, 5, 6, 9, 10)
0.34508325227122005
(1, 5, 

[(0.3514942385018405, (1, 4, 6, 9, 10)),
 (0.3483140356739335, (1, 4, 8, 9, 10)),
 (0.3469578882102964, (1, 6, 8, 9, 10)),
 (0.34667490118495603, (0, 1, 4, 9, 10)),
 (0.34580097696162626, (0, 1, 6, 9, 10)),
 (0.34508325227122005, (1, 5, 6, 9, 10)),
 (0.3443635711764268, (1, 4, 5, 9, 10)),
 (0.3440351608789971, (1, 3, 6, 9, 10)),
 (0.3439352967011313, (0, 1, 2, 9, 10)),
 (0.34388579128162566, (1, 6, 7, 9, 10)),
 (0.3438525037776027, (1, 2, 6, 9, 10)),
 (0.343362604424549, (1, 4, 7, 9, 10)),
 (0.3431028532887376, (1, 2, 4, 9, 10)),
 (0.34309217135533454, (1, 3, 4, 9, 10)),
 (0.34134323636965247, (1, 2, 8, 9, 10)),
 (0.3406926425433441, (0, 1, 7, 9, 10)),
 (0.33961135162135403, (0, 1, 5, 9, 10)),
 (0.3396055672621333, (0, 1, 8, 9, 10)),
 (0.33954706765163467, (1, 5, 8, 9, 10)),
 (0.33920997861088864, (0, 1, 3, 9, 10)),
 (0.3389081628512486, (1, 3, 8, 9, 10)),
 (0.33882186451170304, (1, 7, 8, 9, 10)),
 (0.33731531734938525, (1, 2, 5, 9, 10)),
 (0.3371090739871402, (1, 5, 7, 9, 10)),
 (0.33

## Part 1: Wrapper methods

Within wrapper methods, there are different strategies for selecting the features. Here we will learn about **forward selection**, **backward elimination** and **recursive feature elimination** strategies.

### A. Forward selection

Forward selection is an iterative method in which we start with having no feature in the model. 
In each iteration, we keep adding the feature which best improves our model.

In [7]:
def forward_select(estimator, X, y, k=5):
    # this array holds indicators of whether each feature is currently selected
    selected = np.zeros(X.shape[1]).astype(bool)
    
    # fit and score model based on some subset of features
    score = lambda X_features: clone(estimator).fit(X_features, y).score(X_features, y)  # accurary is the measure
    
    # find indices to selected columns
    selected_indices = lambda: list(np.flatnonzero(selected))
    
    # repeat till k features are selected
    while np.sum(selected) < k:
        # indices to unselected columns
        rest_indices = list(np.flatnonzero(~selected))
    
        # compute model scores with an additional feature
        scores = [score(X[:, selected_indices() + [i]]) for i in rest_indices]
        print('\n%accuracy if adding column:\n   ',
              {i:int(s*100) for i,s in zip(rest_indices,scores)})
        
        # find index within `rest_indices` that points to the most predictive feature not yet selected 
        idx_to_add = rest_indices[np.argmax(scores)]
        print('add column', idx_to_add)
        
        # select this new feature
        selected[idx_to_add] = True
        
    return selected_indices()

support = sorted(forward_select(LinearRegression(), X, y))
print(support)


%accuracy if adding column:
    {0: 1, 1: 15, 2: 5, 3: 0, 4: 1, 5: 0, 6: 3, 7: 3, 8: 0, 9: 6, 10: 22}
add column 10

%accuracy if adding column:
    {0: 25, 1: 31, 2: 25, 3: 22, 4: 22, 5: 22, 6: 23, 7: 23, 8: 25, 9: 26}
add column 1

%accuracy if adding column:
    {0: 32, 2: 31, 3: 31, 4: 31, 5: 31, 6: 32, 7: 31, 8: 32, 9: 33}
add column 9

%accuracy if adding column:
    {0: 33, 2: 33, 3: 33, 4: 34, 5: 33, 6: 34, 7: 33, 8: 33}
add column 6

%accuracy if adding column:
    {0: 34, 2: 34, 3: 34, 4: 35, 5: 34, 7: 34, 8: 34}
add column 4
[1, 4, 6, 9, 10]


### B. Backward elimination

In backward elimination, 
we start with all the features and remove the _least significant_ feature at each iteration,
which improves the performance of the model.

In [8]:
def backward_eliminate(estimator, X, y, k=5):
    # this array holds indicators of whether each feature is currently selected
    selected = np.ones(X.shape[1]).astype(bool)
    
    # fit and score model based on some subset of features
    score = lambda X_features: clone(estimator).fit(X_features, y).score(X_features, y)
    
    # find indices to selected columns
    selected_indices = lambda: list(np.flatnonzero(selected))
    
    # repeat till k features are selected
    while np.sum(selected) > k:
        # Compute model scores with one of the features removed
        scores = [score(X[:, list(set(selected_indices()) - {i})]) for i in selected_indices()]
        print('\n%accuracy if removing column:\n   ',
              {i:int(s*100) for i,s in zip(selected_indices(), scores)})
        
        # Find index that points to the least predictive feature
        idx_to_remove = selected_indices()[np.argmax(scores)]
        print('remove column', idx_to_remove)
        
        # Remove this feature
        selected[idx_to_remove] = False
        
    return selected_indices()

support = sorted(backward_eliminate(LinearRegression(), X, y))
print(support)


%accuracy if removing column:
    {0: 36, 1: 32, 2: 35, 3: 36, 4: 35, 5: 35, 6: 35, 7: 36, 8: 35, 9: 33, 10: 31}
remove column 7

%accuracy if removing column:
    {0: 36, 1: 32, 2: 35, 3: 36, 4: 35, 5: 35, 6: 35, 8: 35, 9: 33, 10: 24}
remove column 0

%accuracy if removing column:
    {1: 32, 2: 35, 3: 35, 4: 35, 5: 35, 6: 35, 8: 35, 9: 33, 10: 24}
remove column 3

%accuracy if removing column:
    {1: 32, 2: 35, 4: 35, 5: 35, 6: 35, 8: 35, 9: 33, 10: 24}
remove column 2

%accuracy if removing column:
    {1: 31, 4: 34, 5: 35, 6: 34, 8: 35, 9: 33, 10: 24}
remove column 5

%accuracy if removing column:
    {1: 31, 4: 34, 6: 34, 8: 35, 9: 33, 10: 23}
remove column 8
[1, 4, 6, 9, 10]


### C. Recursive feature elimination

**Recursive Feature elimination** is an even more greedy algorithm provided by sklearn, 
which finds good performing feature subset with high efficiency. This method has similarity with backward elimination technique. 

The importance of each feature is obtained either through a **`coef_`** attribute 
or through a **`feature_importances_`** attribute.
So in order for recursive feature elimination algorithm in sklearn to work, 
the model is required to provide either of these attributes.

Usually, we start off using a low complexity model and use it as a benchmark for feature selection.

In [9]:
model = LinearRegression()
selector = RFE(model, n_features_to_select=5)
selector.fit(X, y)
print("Num Features:", selector.n_features_)
print("Selected Features:", np.flatnonzero(selector.support_))

Num Features: 5
Selected Features: [1 4 7 8 9]


Then we can transform dataset to include only these features.

In [10]:
X_new = selector.transform(X)
print(X_new.shape)

(1599, 5)


## Part 2: Filter methods

In filter methods, features are ranked in terms of predictiveness one by one, 
as opposed to considering a subset.
They incorporate statistical or information theoretic measures to rank each feature instead of measuring accuracy of a model trained on selected features.

### A. Pearson's χ² test based feature selection

The following cell shows the usage of a feature selector based on Pearson's χ² test.
This constructs the approximate χ² distribution and scores each feature vs 
the label in order to determine which feature is more relevant, 
one at a time, then selects features according to the score.

In [11]:
selector = SelectKBest(chi2, k=5)
selector.fit(X, y)
print('χ² statistic', selector.scores_)
print('Selected indices', selector.get_support(True))

χ² statistic [1.12606524e+01 1.55802891e+01 1.30256651e+01 4.12329474e+00
 7.52425579e-01 1.61936036e+02 2.75555798e+03 2.30432045e-04
 1.54654736e-01 4.55848775e+00 4.64298922e+01]
Selected indices [ 1  2  5  6 10]


We can call the **transform()** method to select those feature columns from dataset and 
store into a new variable **X_selected**.

In [12]:
X_selected = selector.transform(X)
X_selected.shape

(1599, 5)

**selector.transform()** does the same as slicing out these columns.

In [13]:
np.allclose(X_selected, X[:, [1, 2, 5, 6, 10]])

True

Next we will take a closer look at this procedure by implementing one so we can better distinguish different feature selection methods. 
χ² test has many applications.
For feature selection, we utilize χ² statistic to test for dependence of each feature towards determining label.

**Note:** _You do NOT need to learn by heart the details of the computation to work on practices._

#### Step 1: Encode labels into orthogonal vector space

This is also know as **one-hot encoding**, which is applicable to classification problems.
Consider an example where you have defined 3 categories for possible outcomes: A, B and C.
In order for machine learning algorithms to be able to handle this type of data,
we have to convert them into numbers.
One-hot encoding uses a vector $ (y_1, y_2, y_3) $ where 
$$ y_i = \left[ \text{result falls into i}^{th}\text{ category} \right] \in \left\{ 0, 1 \right\} $$
Therefore, one-hot encoding for A, B and C categories becomes (1, 0, 0), (0, 1, 0) and (0, 0, 1) respectively.
This has an advantage over plainly translating A, B and C into 1, 2 and 3 (a.k.a **sparse encoding**)
in a way that orthogonal vectors do not impose assumptions of their order or magnitudes between categories like numbers would.

For example, 3>1 is true, how ever it doesn't mean to imply C>A or C is superior to A in any way.
However this would affect the model's numerical stability.

Therefore one-hot encoding is an widely adopted technique for processing categories. 
Sparse encoding could be used when persisting a dataset in order to save storage space.

**Note:** If you recall _regression in R_ from 8610 (Stat/Math), 
one-hot encoding is how the categorical / nominal variables are encoded as 
independent predictors in the regression formula.


In [14]:
from sklearn.preprocessing import LabelBinarizer
Y = np.array(LabelBinarizer().fit_transform(y))
print(Y.shape)
print(Y[:5])

(1599, 6)
[[0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 0 1 0 0]
 [0 0 0 1 0 0]
 [0 0 1 0 0 0]]


#### Step 2: Compute the [contingency table](https://en.wikipedia.org/wiki/Contingency_table) of observed frequencies

**observed** is a (#classes)-by-(#features) matrix that contains the "number of occurrences" for each combination of feature and classes.

**Note:** You previously saw contingency tables in the 8610 (Stat/Math) class and computed the Chi-Squared (χ²) statistic there.

In [15]:
observed = np.dot(Y.T, X)

#### Step 3: Compute the expected frequencies using marginal frequencies

**expected** has the same shape as **observed** matrix, but represent the expected frequencies in theory.

In [16]:
expected = np.dot(
    Y.mean(axis=0).reshape(1, -1).T, # Mean value for all classes (transposed)
    X.sum(axis=0).reshape(1, -1) # Marginal frequencies for all features
)

#### Step 4: Compute the χ² statistic between **observed** and **expected**

In [17]:
chi_squared = np.sum((observed-expected)**2 / expected, axis=0)
print(chi_squared)

[1.12606524e+01 1.55802891e+01 1.30256651e+01 4.12329474e+00
 7.52425579e-01 1.61936036e+02 2.75555798e+03 2.30432045e-04
 1.54654736e-01 4.55848775e+00 4.64298922e+01]


Compare with sklearn **chi2()**, which returns two arrays containing χ² statistic and p-value, respectively.

In [18]:
chi2_sklearn, pvalue_sklearn = chi2(X, y)
print("Chi-Squared Statistic")
print(chi2_sklearn)

print("P-Values")
print(pvalue_sklearn)

Chi-Squared Statistic
[1.12606524e+01 1.55802891e+01 1.30256651e+01 4.12329474e+00
 7.52425579e-01 1.61936036e+02 2.75555798e+03 2.30432045e-04
 1.54654736e-01 4.55848775e+00 4.64298922e+01]
P-Values
[4.64500416e-02 8.15035154e-03 2.31394417e-02 5.31804675e-01
 9.79968040e-01 3.82728810e-33 0.00000000e+00 1.00000000e+00
 9.99526491e-01 4.72096321e-01 7.42403757e-09]


#### Step 5: Rank features descendingly by χ² statistic

Optionally, then sort these indices so they remain relative order.

In [19]:
support = sorted(np.argsort(-chi_squared)[:5])
print(support)

[1, 2, 5, 6, 10]


#### Step 6: Select these features and transform dataset

In [20]:
X_new = X[:, support]
np.allclose(X_new, X_selected)

True

### A note on the p-value

sklearn not only has provided the **χ² statistic** but also **p-value**,
which is very useful because p-value could tell you quantitively how probable each feature is relevant,
which in turn helps you decide how many and which features are worthwhile to retain.

Here's part of a **χ² distribution** table.
In our dataset, we have (#classes - 1) = 5 degrees of freedom (d.o.f.).

<table cellspacing="2" cellpadding="3" border="1" align="center">
<tbody>
<tr class="col1"><td bgcolor="#009bff">df</td><td bgcolor="#009bff">0.995</td><td bgcolor="#009bff">0.99</td><td bgcolor="#009bff">0.975</td><td bgcolor="#009bff">0.95</td><td bgcolor="#009bff">0.9</td><td bgcolor="#009bff">0.1</td><td bgcolor="#009bff">0.05</td><td bgcolor="#009bff">0.025</td><td bgcolor="#009bff">0.01</td><td bgcolor="#009bff">0.005</td></tr>
<tr class="col1"><td bgcolor="#009bff">1</td><td bgcolor="#cdebfa">0</td><td bgcolor="#edfbfa">0</td><td bgcolor="#cdebfa">0.001</td><td bgcolor="#edfbfa">0.004</td><td bgcolor="#cdebfa">0.016</td><td bgcolor="#edfbfa">2.706</td><td bgcolor="#cdebfa">3.841</td><td bgcolor="#edfbfa">5.024</td><td bgcolor="#cdebfa">6.635</td><td bgcolor="#edfbfa">7.879</td></tr>
<tr class="col1"><td bgcolor="#009bff">2</td><td bgcolor="#edfbfa">0.01</td><td bgcolor="#cdebfa">0.02</td><td bgcolor="#edfbfa">0.051</td><td bgcolor="#cdebfa">0.103</td><td bgcolor="#edfbfa">0.211</td><td bgcolor="#cdebfa">4.605</td><td bgcolor="#edfbfa">5.991</td><td bgcolor="#cdebfa">7.378</td><td bgcolor="#edfbfa">9.21</td><td bgcolor="#cdebfa">10.597</td></tr>
<tr class="col1"><td bgcolor="#009bff">3</td><td bgcolor="#cdebfa">0.072</td><td bgcolor="#edfbfa">0.115</td><td bgcolor="#cdebfa">0.216</td><td bgcolor="#edfbfa">0.352</td><td bgcolor="#cdebfa">0.584</td><td bgcolor="#edfbfa">6.251</td><td bgcolor="#cdebfa">7.815</td><td bgcolor="#edfbfa">9.348</td><td bgcolor="#cdebfa">11.345</td><td bgcolor="#edfbfa">12.838</td></tr>
<tr class="col1"><td bgcolor="#009bff">4</td><td bgcolor="#edfbfa">0.207</td><td bgcolor="#cdebfa">0.297</td><td bgcolor="#edfbfa">0.484</td><td bgcolor="#cdebfa">0.711</td><td bgcolor="#edfbfa">1.064</td><td bgcolor="#cdebfa">7.779</td><td bgcolor="#edfbfa">9.488</td><td bgcolor="#cdebfa">11.143</td><td bgcolor="#edfbfa">13.277</td><td bgcolor="#cdebfa">14.86</td></tr>
<tr class="col1"><td bgcolor="#009bff">5</td><td bgcolor="#cdebfa">0.412</td><td bgcolor="#edfbfa">0.554</td><td bgcolor="#cdebfa">0.831</td><td bgcolor="#edfbfa">1.145</td><td bgcolor="#cdebfa">1.61</td><td bgcolor="#edfbfa">9.236</td><td bgcolor="#cdebfa">11.07</td><td bgcolor="#edfbfa">12.833</td><td bgcolor="#cdebfa">15.086</td><td bgcolor="#edfbfa">16.75</td></tr>
<tr class="col1"><td bgcolor="#009bff">6</td><td bgcolor="#edfbfa">0.676</td><td bgcolor="#cdebfa">0.872</td><td bgcolor="#edfbfa">1.237</td><td bgcolor="#cdebfa">1.635</td><td bgcolor="#edfbfa">2.204</td><td bgcolor="#cdebfa">10.645</td><td bgcolor="#edfbfa">12.592</td><td bgcolor="#cdebfa">14.449</td><td bgcolor="#edfbfa">16.812</td><td bgcolor="#cdebfa">18.548</td></tr>
<tr class="col1"><td bgcolor="#009bff">7</td><td bgcolor="#cdebfa">0.989</td><td bgcolor="#edfbfa">1.239</td><td bgcolor="#cdebfa">1.69</td><td bgcolor="#edfbfa">2.167</td><td bgcolor="#cdebfa">2.833</td><td bgcolor="#edfbfa">12.017</td><td bgcolor="#cdebfa">14.067</td><td bgcolor="#edfbfa">16.013</td><td bgcolor="#cdebfa">18.475</td><td bgcolor="#edfbfa">20.278</td></tr>
<tr class="col1"><td bgcolor="#009bff">8</td><td bgcolor="#edfbfa">1.344</td><td bgcolor="#cdebfa">1.646</td><td bgcolor="#edfbfa">2.18</td><td bgcolor="#cdebfa">2.733</td><td bgcolor="#edfbfa">3.49</td><td bgcolor="#cdebfa">13.362</td><td bgcolor="#edfbfa">15.507</td><td bgcolor="#cdebfa">17.535</td><td bgcolor="#edfbfa">20.09</td><td bgcolor="#cdebfa">21.955</td></tr>
<tr class="col1"><td bgcolor="#009bff">9</td><td bgcolor="#cdebfa">1.735</td><td bgcolor="#edfbfa">2.088</td><td bgcolor="#cdebfa">2.7</td><td bgcolor="#edfbfa">3.325</td><td bgcolor="#cdebfa">4.168</td><td bgcolor="#edfbfa">14.684</td><td bgcolor="#cdebfa">16.919</td><td bgcolor="#edfbfa">19.023</td><td bgcolor="#cdebfa">21.666</td><td bgcolor="#edfbfa">23.589</td></tr>
<tr class="col1"><td bgcolor="#009bff">10</td><td bgcolor="#edfbfa">2.156</td><td bgcolor="#cdebfa">2.558</td><td bgcolor="#edfbfa">3.247</td><td bgcolor="#cdebfa">3.94</td><td bgcolor="#edfbfa">4.865</td><td bgcolor="#cdebfa">15.989</td><td bgcolor="#edfbfa">18.307</td><td bgcolor="#cdebfa">20.483</td><td bgcolor="#edfbfa">23.209</td><td bgcolor="#cdebfa">25.188</td></tr>
</tbody>
</table>



Our null hypothesis is that a class labels is independent of a feature. 
Since here we used **χ² statistic** for testing what the null hypothesis has claimed : **independence between features and labels**.
We can choose `p-value` $>2.5\%$ as the critical level at which the null hypothesis **can not be rejected**,
and reject the hypothesis otherwise.
This is effectively saying that, if the feature is more that 10% chance to be independent of the class, then it is not a good predictor.
The alternative hypothesis is, therefore, that the feature is probable predictor of the class, i.e., the class is dependent on the feature.
The following table would summarize our conclusion based on this criterion:

<table>
<tr><td><strong>feature idx</strong></td><td>0</td><td>1</td><td>2</td><td>3</td><td>4</td><td>5</td><td>6</td><td>7</td><td>8</td><td>9</td><td>10</td></tr>
<tr><td><strong>χ²</strong></td><td>11.3</td><td>15.6</td><td>13.0</td><td>4.12</td><td>0.752</td><td>162</td><td>2.76e+03</td><td>2.30e-04</td><td>0.155</td><td>4.56</td><td>46.4</td></tr>
<tr><td><strong>p-value</strong></td><td>~5%</td><td>~0.5%</td><td>~1%</td><td>90%~95%</td><td>100%</td><td>0%</td><td>0%</td><td>100%</td><td>100%</td><td>90%~95%</td><td>0%</td></tr>
<tr><td><strong>interpretation</strong></td><td>independent</td><td><strong>dependent</strong></td><td><strong>dependent</strong></td><td>independent</td><td>independent</td><td><strong>dependent</strong></td><td><strong>dependent</strong></td><td>independent</td><td>independent</td><td>independent</td><td><strong>dependent</strong></td></tr>
</table>

So we should select feature 1, 2, 5, 6 and 10.

<!-- future work:
sklearn learn can help make this process more precise and easy than looking up the **χ² distribution** table.
Following cells prints the all the p-values:
-->

Please review the documentation of the chi2 function from `sklearn.feature_selection`.

In [21]:
help(chi2)

Help on function chi2 in module sklearn.feature_selection._univariate_selection:

chi2(X, y)
    Compute chi-squared stats between each non-negative feature and class.
    
    This score can be used to select the n_features features with the
    highest values for the test chi-squared statistic from X, which must
    contain only non-negative features such as booleans or frequencies
    (e.g., term counts in document classification), relative to the classes.
    
    Recall that the chi-square test measures dependence between stochastic
    variables, so using this function "weeds out" the features that are the
    most likely to be independent of class and therefore irrelevant for
    classification.
    
    Read more in the :ref:`User Guide <univariate_feature_selection>`.
    
    Parameters
    ----------
    X : {array-like, sparse matrix} of shape (n_samples, n_features)
        Sample vectors.
    
    y : array-like of shape (n_samples,)
        Target vector (class labels).



--- 

Here's how to generate χ² table, which may come in handy now and then.

In [22]:
def chi2_table(degree_of_freedoms):
    from scipy.stats import chi2 as chi2_distribution
    pvalue = np.array([0.995, 0.99, 0.975, 0.95, 0.90, 0.10, 0.05, 0.025, 0.01, 0.005])
    return pd.DataFrame(chi2_distribution.isf(
            # isf(p) = inverse(1-cdf)(p) which takes p-value returns chi square value
            #     where cdf is short for cumulative distribution function
        pvalue, np.expand_dims(degree_of_freedoms, 1)),
        columns = pvalue, index = degree_of_freedoms)

chi2_table(range(5, 12))

,0.995,0.990,0.975,0.950,0.900,0.100,0.050,0.025,0.010,0.005
5,0.411742,0.554298,0.831212,1.145476,1.610308,9.236357,11.070498,12.832502,15.086272,16.749602
6,0.675727,0.872090,1.237344,1.635383,2.204131,10.644641,12.591587,14.449375,16.811894,18.547584
7,0.989256,1.239042,1.689869,2.167350,2.833107,12.017037,14.067140,16.012764,18.475307,20.277740
8,1.344413,1.646497,2.179731,2.732637,3.489539,13.361566,15.507313,17.534546,20.090235,21.954955
9,1.734933,2.087901,2.700389,3.325113,4.168159,14.683657,16.918978,19.022768,21.665994,23.589351
10,2.155856,2.558212,3.246973,3.940299,4.865182,15.987179,18.307038,20.483177,23.209251,25.188180
11,2.603222,3.053484,3.815748,4.574813,5.577785,17.275009,19.675138,21.920049,24.724970,26.756849


--- 
### B. ANOVA F-value based feature selection

The following cell shows the usage of a feature selector based on ANOVA F-value and Pearson's correlation.
This calculates Pearson's correlation of each feature vs the label in order to determine which feature is more relevant, 
one at a time, then selects features according to the ANOVA F-value derived from Pearson's correlation.

See also: https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.pearsonr.html

**Note:** You previously saw the ANOVA and MANOVA within the 8610 (Stat/Math) course.

In [23]:
selector = SelectKBest(f_regression, k=5)
selector.fit(X, y)
print('score', selector.scores_)
print('Selected indices', selector.get_support(True))

score [2.49600375e+01 2.87444450e+02 8.62577262e+01 3.01183699e-01
 2.69856084e+01 4.10850227e+00 5.66578176e+01 5.04052231e+01
 5.34046221e+00 1.07740433e+02 4.68267011e+02]
Selected indices [ 1  2  6  9 10]


Next we will take a closer look at this procedure to gain a more solid understanding.

**Note:** _You do NOT need to learn by heart the details of the computation to work on practices._

#### Step 1: Compute cross correlation

$$r_j = \frac{\sigma_{X_j y}}{\sigma_{X_j} \sigma_y} = \frac{(y-\bar y)^T (X_j-\bar {X_j})}{\lVert X_j-\bar {X_j}\rVert \cdot \lVert y-\bar y\rVert}$$


In [24]:
from sklearn.preprocessing import scale
X_centered = scale(X.astype('float'), with_std = False)
y_centered = scale(y.astype('float'), with_std = False)
corr = np.dot(y_centered, X_centered) / np.linalg.norm(X_centered, axis = 0) / np.linalg.norm(y_centered)
print(corr)

[ 0.12405165 -0.39055778  0.22637251  0.01373164 -0.12890656 -0.05065606
 -0.18510029 -0.17491923 -0.05773139  0.25139708  0.47616632]


In [25]:
from scipy.stats import pearsonr
print([pearsonr(X[:,i], y)[0] for i in range(X.shape[1])])

[0.12405164911322433, -0.3905577802640073, 0.22637251431804137, 0.013731637340066292, -0.12890655993005268, -0.050656057244276395, -0.18510028892653782, -0.17491922778334876, -0.05773139120538214, 0.25139707906926134, 0.4761663240011359]


#### Step 2: Compute ANOVA F-value

This is a F-test for correlation coefficients.
Read more [here](https://onlinecourses.science.psu.edu/stat501/lesson/2/2.6).
Similar to the way that χ² test is provided by sklearn, [f_regression()](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_regression.html#sklearn.feature_selection.f_regression) also supplied p-value for quantitatively assessing how relevant each feature is.

$$ F = t^2 = \left( \frac{r\sqrt{n-2}}{\sqrt{1-r^2}} \right) ^2 = \frac{r^2/1 }{(1-r^2)/(n-2)} $$

In [26]:
corr2 = corr ** 2
Fvalue = corr2 / (1 - corr2) * (y.shape[0] - 2)
print(Fvalue)

[2.49600375e+01 2.87444450e+02 8.62577262e+01 3.01183699e-01
 2.69856084e+01 4.10850227e+00 5.66578176e+01 5.04052231e+01
 5.34046221e+00 1.07740433e+02 4.68267011e+02]


Compare to f_regression().

In [27]:
Fvalue_sklearn, pvalue2_sklearn = f_regression(X,y)
print(Fvalue_sklearn)

[2.49600375e+01 2.87444450e+02 8.62577262e+01 3.01183699e-01
 2.69856084e+01 4.10850227e+00 5.66578176e+01 5.04052231e+01
 5.34046221e+00 1.07740433e+02 4.68267011e+02]


#### Step 3: Select these features and transform dataset

In [28]:
support = sorted(np.argsort(-Fvalue)[:5])
print(support)

[1, 2, 6, 9, 10]


### More on F-value and p-value

F-test can be used as hypothesis testing for a ratio of two χ² distributions.
The F-test for correlation coefficients is derived from testing a ratio between regression sum square (SSR)
and error sum square (SSE).

$$ SSR = \sum _{i=1}^n {\left(\hat {y_i} - \bar y \right)^2}$$

$$ SSE = \sum _{i=1}^n {\left(\hat {y_i} - y_i \right)^2}$$

$$ F  = \frac{SSR/(v-1) }{SSE/(n-2)} $$

With correlation coefficients, this is 

$$ F = \frac{r^2/1 }{(1-r^2)/(n-2)} $$

with degree of freedom 1 and (n-2) respectively.

Its null hypothesis claims r = 0, i.e. the independence of each feature and label.
We could theoretically make hypothesis testing using an F-distribution table.

Here's how to generate F table.

In [29]:
def f_table(alpha, v1, v2):
    from scipy.stats import f as f_distribution
    v1 = np.array(list(v1)); v2 = np.array(list(v2))
    return pd.DataFrame(f_distribution.isf(alpha, v1[np.newaxis, ...], v2[..., np.newaxis]),
        columns = v1, index = v2)

import itertools
f_table(0.1, range(1, 9), itertools.chain(range(1,10), range(10, 260, 20)))

,1,2,3,4,5,6,7,8
1,39.863458,49.500000,53.593245,55.832961,57.240077,58.204416,58.905953,59.438981
2,8.526316,9.000000,9.161790,9.243416,9.292626,9.325530,9.349081,9.366770
3,5.538319,5.462383,5.390773,5.342644,5.309157,5.284732,5.266195,5.251671
4,4.544771,4.324555,4.190860,4.107250,4.050579,4.009749,3.978966,3.954940
5,4.060420,3.779716,3.619477,3.520196,3.452982,3.404507,3.367899,3.339276
6,3.775950,3.463304,3.288762,3.180763,3.107512,3.054551,3.014457,2.983036
7,3.589428,3.257442,3.074072,2.960534,2.883344,2.827392,2.784930,2.751580
8,3.457919,3.113118,2.923796,2.806426,2.726447,2.668335,2.624135,2.589349
9,3.360303,3.006452,2.812863,2.692680,2.610613,2.550855,2.505313,2.469406
10,3.285015,2.924466,2.727673,2.605336,2.521641,2.460582,2.413965,2.377150


Alternatively, directly compute p-value.

In [30]:
from scipy.stats import f as f_distribution
f_distribution.sf(Fvalue, 1, y.shape[0] - 2)

array([6.49563501e-07, 2.05171481e-59, 4.99129525e-20, 5.83218013e-01,
       2.31338265e-07, 4.28339795e-02, 8.62170342e-14, 1.87495665e-12,
       2.09627787e-02, 1.80208845e-24, 2.83147697e-91])

Compare to those obtained from f_regression().

In [31]:
pvalue2_sklearn

array([6.49563501e-07, 2.05171481e-59, 4.99129525e-20, 5.83218013e-01,
       2.31338265e-07, 4.28339795e-02, 8.62170342e-14, 1.87495665e-12,
       2.09627787e-02, 1.80208845e-24, 2.83147697e-91])

Print in percentages perhaps makes it easier to read.

In [32]:
np.round(pvalue2_sklearn * 100)

array([ 0.,  0.,  0., 58.,  0.,  4.,  0.,  0.,  2.,  0.,  0.])

### Recap

We just went through **χ²** and **F-value** based feature selection in detail because 
these two different methods work better on classification and regression respectively.
The general steps for doing feature selection with sklearn was:

1. Choose a feature scoring method
2. Initialize a feature selector
3. Fit feature selector on the data

Other feature selection methods provided by sklearn include:

* Classification: chi2, f_classif, mutual_info_classif
* Regresssion: f_regression, mutual_info_regression



# Conclusion

In this lab, we learned about:

* Wrapper methods
    * Forward selection
    * Backward elimination
    * Recursive feature elimination
    
 
* Filter methods
    * Pearson's χ²
    * ANOVA F-value


And once again,
1. Wrapper methods are usually computationally expensive
2. Greedy algorithms don't necessary provide the optimal solution, which may be good becuase it makes them less prone to overfitting.